In [19]:
import csv
import numpy as np
import cv2
import face_recognition as fr
import os
from datetime import datetime



In [20]:
path="images" # now images are in path

In [21]:
mylist=os.listdir(path)
mylist

['dhoni.jpg', 'mamooty.jpg', 'mohanlal.jpg', 'ronaldo.jpeg', 'sachin.jpeg']

In [22]:
imgs=[]  # for input
classname=[] #for output
for img in mylist:
    imgpath=os.path.join(path,img) # extracting path of each image
    # print(imgpath)
    current_img=cv2.imread(imgpath) # reading each image
    # print(current_img)
    imgs.append(current_img) #addimg images to input
    classname.append(img.split(".")[0]) # extracting only name from imgs by removing .jpeg,.jpg and append to output


In [23]:
def faceencoding(images):
    encodelist=[]
    for img in images:
        face_in_frame=fr.face_locations(img)
        encode=fr.face_encodings(img,face_in_frame)[0]
        encodelist.append(encode)
    return encodelist

In [24]:
encode_list_known=faceencoding(imgs) #now all images are encoded


In [25]:


data_set = set()

Video = cv2.VideoCapture(0)

while True:
    success, image = Video.read()
    if not success:
        break

    conv_img1 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    face_in_frame = fr.face_locations(conv_img1)
    face_encoded = fr.face_encodings(conv_img1, face_in_frame)

    for enc_face, faceloc in zip(face_encoded, face_in_frame):
        matches = fr.compare_faces(encode_list_known, enc_face)
        face_dis = fr.face_distance(encode_list_known, enc_face)

        matchindex = np.argmin(face_dis)
        if matches[matchindex]:
            name = classname[matchindex]
            
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S") # read current time


            data_set.add((name,current_time))
        else:
            name = "unknown"

        y1, x2, y2, x1 = faceloc
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 3)
        cv2.putText(image, name, (x1, y1 - 10), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Face Recognition", image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

Video.release()
cv2.destroyAllWindows()

print("Recognized people in this session:", data_set)


Recognized people in this session: {('ronaldo', '2025-06-18 19:49:21'), ('ronaldo', '2025-06-18 19:49:22'), ('ronaldo', '2025-06-18 19:49:24'), ('ronaldo', '2025-06-18 19:49:23'), ('ronaldo', '2025-06-18 19:49:20')}


In [26]:
latest_entry = {}

for name, timestamp in data_set:
    if name not in latest_entry or timestamp > latest_entry[name]:
        latest_entry[name] = timestamp

# Convert back to set of (name, timestamp)
final_set = {(name, time) for name, time in latest_entry.items()}

print(final_set)


{('ronaldo', '2025-06-18 19:49:24')}


In [27]:
with open("recognized_people11.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Name", "Timestamp"])  # header
    writer.writerows(final_set)

print("CSV file saved: recognized_people.csv")

CSV file saved: recognized_people.csv
